In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os
from pprint import pprint

import geopandas as gpd
import teotil3 as teo
from shapely import force_2d

# Process NIBIO regines

NIBIO have used a different set of regines to the ones I originally downloaded. Ideally we should both update to use the latest version, but this is difficult for NIBIO, so I will switch to using their dataset instead. 

NIBIO originally downloaded a shapefile rather than a geodatabase. This notebook restructures the shapefile into the format expected by TEOTIL. It's essentially a temporary replacement for `teo.preprocessing.read_raw_regine_data`, but customised to work with the NIBIO shapefile.

In [4]:
# Original NVE data
data_fold = f"/home/jovyan/shared/common/teotil3/core_data"
nve_gdb = os.path.join(data_fold, "hydrology", "NVEData.gdb")

# NIBIO data
shp_path = r"/home/jovyan/shared/common/teotil3/core_data/hydrology/regines_nibio_dec_2023/Nedborfelt_RegineEnhet.shp"

# Output geopackage
nib_gpkg = os.path.join(data_fold, "hydrology", "nibio_regines.gpkg")

In [5]:
# Read original and NIBIO datasets
reg_gdf = gpd.read_file(nve_gdb, driver="fileGDB", layer="RegineEnhet")
nib_gdf = gpd.read_file(shp_path)

# Get column name mapping and check it looks OK
names_dict = {
    shp: gdb
    for shp, gdb in zip(nib_gdf.columns, reg_gdf.columns)
    if gdb not in ("SHAPE_Length", "SHAPE_Area", "geometry")
}
pprint(names_dict)

# Rename
nib_gdf.rename(columns=names_dict, inplace=True)

# Convert to PolyZ to Poly
nib_gdf["geometry"] = nib_gdf["geometry"].apply(lambda geom: force_2d(geom))

# Process to match format expected by TEOTIL
nib_gdf.rename(
    {
        "vassdragsnummer": "regine",
        "nedborfeltOppstromAreal_km2": "upstr_a_km2",
    },
    axis="columns",
    inplace=True,
)

# Remove Svalbard
nib_gdf["vassom"] = nib_gdf["regine"].str.split(".", n=1).str[0].astype(int)
nib_gdf = nib_gdf.query("vassom < 400").copy()
nib_gdf["vassom"] = nib_gdf["vassom"].apply(lambda x: f"{x:03}")

# Calculate polygon area
nib_gdf["a_cat_poly_km2"] = nib_gdf.to_crs({"proj": "cea"})["geometry"].area / 1e6

# Get columns of interest
reg_cols = [
    "regine",
    "vassom",
    "a_cat_poly_km2",
    "upstr_a_km2",
    "geometry",
]
nib_gdf = nib_gdf[reg_cols]

# Sort by 'regine' and reset index
nib_gdf.sort_values("regine", inplace=True)
nib_gdf.reset_index(inplace=True, drop=True)

nib_gdf.head()

{'Q3060lskm2': 'QNormal3060_lskm2',
 'Q6190lskm2': 'QNormalr6190_lskm2',
 'enhAreal': 'regineAreal_km2',
 'hierarki': 'elvNavnHierarki',
 'navnLokal': 'lokalNavn',
 'navnNbfHav': 'nedborfeltTilHav',
 'navnNedbf': 'overordnetNedborfeltNavn',
 'nbfHavNr': 'nedbfeltHavVassdragNr',
 'niva': 'hierarkiNivaRegine',
 'objType': 'objektType',
 'pktNavnFra': 'punktNavnFra',
 'regineQ': 'QNormalRegine_Mm3Aar',
 'statOmrNr': 'statistikkomrNr',
 'tilPktNavn': 'punktNavnTil',
 'totAreal': 'nedborfeltOppstromAreal_km2',
 'totTilsig': 'QNedborfeltOppstrom_Mm3Aar',
 'vassdragNr': 'vassdragsnummer',
 'vnrOFelt': 'nedborfeltVassdragNrOverordnet'}


,regine,vassom,a_cat_poly_km2,upstr_a_km2,geometry
0,001.10,001,1.442790,0.00,"POLYGON ((297006.830 6543966.950, 297169.290 6..."
1,001.1A1,001,1.432479,777.90,"POLYGON ((297505.440 6543157.790, 297543.100 6..."
2,001.1A20,001,0.340160,777.90,"POLYGON ((297770.368 6543429.036, 297787.114 6..."
3,001.1A2A,001,17.647822,58.96,"POLYGON ((299678.370 6544460.320, 299667.220 6..."
4,001.1A2B,001,41.298255,41.30,"POLYGON ((303353.460 6552989.330, 303341.620 6..."


In [6]:
teo.io.geodataframe_to_geopackage(nib_gdf, nib_gpkg, "RegineEnhetNibio")